#### TODO: Descobrir porquê get_test_case() retorna duas linhas com test_case vazio.

# Importação

In [126]:
import os
import glob
import pandas as pd

from typing import List, Dict, TextIO, DefaultDict
from collections import defaultdict


# Caminho onde se localiza a pasta do equipamento

In [112]:
path_logs = r'/mnt/c/Users/mathe/Documents/AIML_logs/treinamento_aiml/logs'

# Criação do dataframe com as colunas necessárias

In [113]:
logs = pd.DataFrame(columns=['equipment_type', 'run_id', 'test_case', 'timestamp', 'baud_rate', 'slot', 'serial_port', 'tx_data', 'rx_data', 'response_time',
                            'command_protocol', 'command_name'])

# Funções

In [114]:
def get_equipment_type(path: str) -> List[str]:
    '''
    Returns a list with equipment's types.
    Args:
        path (str): Logs path.
    Returns:
        List (str): List with equipment's types.
    '''
    
    equipment_types = []
    entries = os.listdir(path)
    for entry in entries:
        equipment_types.append(entry)
    return equipment_types

In [115]:
def get_run_ids(path: str) -> Dict[str, List[str]]:
    '''
    Returns a dict with equipment's types as keys and a list of run_ids as values.
    Args:
        path (str): Logs path.
    Returns:
        Dict (str, List[str]): Dict with equipment's types as keys and a list of run_ids as values.
    '''
    
    run_ids = defaultdict(list)
    for equipment in logs['equipment_type'].unique():
        entries = os.listdir(f'{path_logs}/{equipment}')
        for entry in entries:
            run_ids[equipment].append(entry)
    return run_ids

In [116]:
def get_test_case(path: str) -> Dict[str, List[str]]:
    '''
    Returns a dict with run_ids as keys and a list of test_cases names as values.
    Args:
        path (str): Logs path.
    Returns:
        Dict (str, List[str]): Dict with run_ids as keys and a list of test_cases names as values.
    '''
    
    test_cases = {}
    for equipment in logs['equipment_type'].unique():
        for run_id in logs['run_id'].unique():
            files = next(os.walk(f'{path_logs}/{equipment}/{run_id}'))[2]
            test_cases[run_id] = [file[10:-4] for file in files if file.endswith('.txt', -4)]
    return test_cases

In [117]:
def open_file(path: str, eq_type: str, run_id: str, test_case: str) -> TextIO:
    '''
    Open a test_case_*.txt file by receiving the path of the equipments, equipment type, run_id and the number of the test case as arguments. Returns the file.
    Args:
        path (str): path where each equipment is located.
        eq_type (str): equipment_type.
        run_id (str): run_id.
        test_case (str): number of the test_case file.
    Returns:
        file (TextIO): the opened test_case file.
    '''
    
    file = open(f'{path}/{eq_type}/{run_id}/test_case_{test_case}.txt', 'r', encoding='cp860')
    return file

In [118]:
def print_lines(path: str, eq_type: str, run_id: str, test_case: str) -> None:
    '''
    Uses the open_file() to open a test_case_*.txt and print its contents.
    Args:
        path (str): path where each equipment is located.
        eq_type (str): equipment_type.
        run_id (str): run_id.
        test_case (str): number of the test_case file.
    '''

    file = open_file(path, eq_type, run_id, test_case)
    [print(line) for line in file.readlines()]
    file.close()

In [119]:
def get_timestamp(path: str) -> DefaultDict[str, List[str]]:
    '''
    Get the timestamp of every line in the test case text file that has a "TX" signal.
    Args:
        path (str): path where each equipment is located.
    Returns:
        timestamp (DefaultDict[str, List[str]]): a dictionary where the keys are the test cases and the values are a list of timestamps.
    '''

    timestamp = defaultdict(list)
    for equipment in logs['equipment_type'].unique():
        for run_id in logs['run_id'].unique():
            for test_case in logs['test_case'].unique():
                file = open_file(path, equipment, run_id, test_case)
                while True:
                    line = file.readline()
                    if 'TX' in line or 'OPTICALTX' in line:
                        timestamp[test_case].append(line[0:25])
                    if not line:
                        break
                file.close()
    return timestamp

# Aplicação das funções

In [120]:
logs['equipment_type'] = get_equipment_type(path_logs)

In [121]:
logs['run_id'] = logs['equipment_type'].map(get_run_ids(path_logs))

In [122]:
logs = logs.explode('run_id')
logs

,equipment_type,run_id,test_case,timestamp,baud_rate,slot,serial_port,tx_data,rx_data,response_time,command_protocol,command_name
0,8721,220818_171756_851_8721_win,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,8721,220905_220107_851_8721_linux,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [123]:
logs['test_case'] = logs['run_id'].map(get_test_case(path_logs))
logs = logs.explode('test_case')
logs = logs.reset_index(drop=True)
logs

,equipment_type,run_id,test_case,timestamp,baud_rate,slot,serial_port,tx_data,rx_data,response_time,command_protocol,command_name
0,8721,220818_171756_851_8721_win,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8721,220818_171756_851_8721_win,10000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8721,220818_171756_851_8721_win,10001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8721,220818_171756_851_8721_win,10002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8721,220818_171756_851_8721_win,10005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1699,8721,220905_220107_851_8721_linux,9995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1700,8721,220905_220107_851_8721_linux,9996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1701,8721,220905_220107_851_8721_linux,9997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1702,8721,220905_220107_851_8721_linux,9998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [124]:
logs = logs.drop(logs.query('test_case == ""').index) # Apaga a linha com test_case nulo. RESOLVER O TODO!!!!!

In [125]:
logs['timestamp'] = logs['test_case'].map(get_timestamp(path_logs))
logs = logs.explode('timestamp')
logs = logs.reset_index(drop=True)
logs

,equipment_type,run_id,test_case,timestamp,baud_rate,slot,serial_port,tx_data,rx_data,response_time,command_protocol,command_name
0,8721,220818_171756_851_8721_win,10000,[19/08/2022 04:53:39.535],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8721,220818_171756_851_8721_win,10000,[19/08/2022 04:53:39.570],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8721,220818_171756_851_8721_win,10000,[19/08/2022 04:53:39.729],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8721,220818_171756_851_8721_win,10000,[19/08/2022 04:53:40.301],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8721,220818_171756_851_8721_win,10000,[19/08/2022 04:53:40.676],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
326905,8721,220905_220107_851_8721_linux,9999,[05/09/2022 23:14:24.675],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
326906,8721,220905_220107_851_8721_linux,9999,[05/09/2022 23:14:25.605],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
326907,8721,220905_220107_851_8721_linux,9999,[05/09/2022 23:14:25.999],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
326908,8721,220905_220107_851_8721_linux,9999,[05/09/2022 23:14:26.916],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
